In [1]:
include("optimization_library.jl");

# Exercise 8.1: Optimal Path for Santa

<img src="figures/santa-claus.png" width="300">

In [2]:
# ================================================================================================
# vector c, matrix A and vector b

c = [4857; 3281; 7891; 7891; 7312; 7312; 9892; 9892]

A = [1 1 0 0 0 0 0 0;
    0 1 1 -1 0 0 1 -1;
    1 0 -1 1 1 -1 0 0;
    0 0 0 0 -1 1 -1 1]

d_eu = 78
d_usa = 60
d_bz = 40
supply = d_eu + d_usa + d_bz

b = [supply; d_eu; d_usa; d_bz]
# ================================================================================================

4-element Vector{Int64}:
 178
  78
  60
  40

In [3]:
# ================================================================================================
# 1. Implement the objective function, its gradient and Hessian.
# 2. Implement the functions g(x) for the inequality constraints, their gradients and Hessians.
nx = size(A,2)

f(x) = c' * x
df(x) = c
Hf(x) = zeros(nx,nx)

g = Vector(undef, nx)
dg = Vector(undef, nx)
Hg = Vector(undef, nx)
    
for i in 1:nx # allow only positive quantities
    g[i] = x -> -x[i]
    dg[i] = x -> begin d = zeros(nx); d[i] = -1; return d end
    Hg[i] = x -> zeros(nx, nx)
end

In [4]:
# ================================================================================================
# Apply the function ConstraintElimination_ipm() from the optimization library
x_best, trace = ConstraintElimination_ipm(f,df,Hf,A,b,g=g,dg = dg, Hg = Hg, eps_barrier = 0.00001, barrier_increase = 10.0);
# ================================================================================================

10.943851197933643


In [5]:
x_best

8-element Vector{Float64}:
 99.99999911788571
 78.00000088211425
  9.360653585588352e-8
  1.4032828854482204e-7
  6.059718593576235e-8
 39.9999992252047
  4.7187068474841e-8
  8.825795703160111e-7

In [6]:
println("average distance per package: ",round((c'*x_best)/supply,digits=2)," km")

average distance per package: 5809.54 km


## How to use the SCS solver

We will now learn how to use the SCS solver using the JuMP package. The JuMP package provides macros that allow to access some functions of the solver in a very convenient way.

In [7]:
using JuMP
using SCS

In [8]:
# ================================================================================================
# Uncomment the following lines to solve the problem with the SCS solver.
model = Model(SCS.Optimizer)
@JuMP.variable(model, x[1:nx])
@JuMP.constraint(model, A * x .== b)
@JuMP.constraint(model,  -x .<= zeros(nx))
@JuMP.objective(model, Min, c'*x)
JuMP.optimize!(model)
# ================================================================================================

------------------------------------------------------------------
	       SCS v3.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 8, constraints m: 12
cones: 	  z: primal zero / dual free vars: 4
	  l: linear vars: 8
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 24, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|2.00e+005 9.89e+003 8.56e+009 -4.28e+009 1.00e-001 4.46e-003 
   225|6.97e-005 1.14e-004 7.22e-001 1.03e+006 2.83e+0

In [9]:
# ================================================================================================
# Uncomment the following line to retrieve the minimizer. Compare the solution to the result from the 
# ConstraintElimination_ipm() function.
sol = JuMP.value.(x)
# ================================================================================================

8-element Vector{Float64}:
 100.00000230135663
  77.99999770467177
  -1.6398455005371797e-5
   1.568723962887483e-5
   3.290514205987734e-5
  40.000067325469054
   1.6746026859733684e-5
  -1.7637593277805285e-5

In [10]:
abs.(sol-x_best) .< 0.01

8-element BitVector:
 1
 1
 1
 1
 1
 1
 1
 1

In [11]:
@assert sum(abs.(sol-x_best) .< 0.01) == length(x_best)

# Exercise 8.2: Truss Structure

In [14]:
using Luxor
# J: Joints
# B: Beams
# Jc: Cantilever joints
# Jl: Joint load
# A: Beam cross-sectional area
function drawTruss(J, B, Jc, Jl, A)
    scale = 40
    function pos(j)
        return Point(J[j][1]*scale - 400, -J[j][2]*scale)
    end
    @pdf begin
        background("white")
        
        # Draw beams
        setcolor("black")
        for b=1:nB
            if A[b] <= 1
                continue
            end
            setline(sqrt(A[b]))
            line(pos(B[b][1]), pos(B[b][2]), :stroke)
        end
        
        # Draw road
        setcolor("blue")
        setline(5)
        line(Point(-400,0), Point(-400 + max_x*scale,0), :stroke)
        
        # Draw joints
        setcolor("red")
        for j = 1:length(J)
            if j in Jc
                circle(pos(j), 8, :fill)
            end
        end
        
        # Draw load arrows
        setcolor("green")
        for j = 1:length(J)
            if LinearAlgebra.norm(Jl[j]) <= 0.1
                continue
            end
            Luxor.arrow(pos(j), pos(j) - (Point(Jl[j][1], Jl[j][2])/500), linewidth=5)
        end
    end 1000 400
end

drawTruss (generic function with 1 method)


(process:33568): GLib-GIO-WARNING **: 18:48:00.197: Failed to open application manifest `C:\Windows\SystemApps\Microsoft.MicrosoftEdge_8wekyb3d8bbwe\AppxManifest.xml' for package #42 (`Microsoft.MicrosoftEdge_44.22621.2134.0_neutral__8wekyb3d8bbwe'): error code 0x2


In [15]:
# Joints. Each is a two-vector with the x-y coordinate
J = [[x,y] for x=0:1:17, y=-2:1:4]
nJ = length(J) # number of joints

# Constructing vector of beams of lenght 1 m. Each beam corresponds to a tuple with the indeces of the joints the beam is connecting
# Connect all joints that are in a radius < 2m.
rad = 2.0
B = [(i,j) for i=1:nJ, j = 1:nJ if j < i && LinearAlgebra.norm(J[i]-J[j]) < rad]

# number of beams
nB = length(B)

# Vector storing indices of beams connected to each joint
# Beams connected to each joint
Bj = [[b for b=1:nB if B[b][1] == j || B[b][2] == j] for j=1:nJ];

# number of joints along x-axis
max_x = maximum([J[i][1] for i=1:nJ])

# Defining cantilever joints as the joints that are on both sides of the bridge and below or equal to the street level
Jc = [i for i=1:nJ if ((J[i][1] == 0 || J[i][1] == max_x) && J[i][2] <= 0)];

# If you want to have two additional cantilever points in the middle of the river uncomment the following line.
# Jc = [i for i=1:nJ if (((J[i][1] == 0 || J[i][1] == max_x) && J[i][2] <= 0) || ((J[i][1] == 6 || J[i][1] == 7) && J[i][2] == 0))];

nC = length(Jc) # number of cantilever points

# external load excerted on each joint at street level --> try also other loads!
load = 2000

# External loads excerted on the joints at street level (y-coordinate equals zero)
Jl = [J[j][2] == 0 ? [0.0, -load*9.81] : [0.0,0.0] for j=1:nJ];

In [16]:
# Initialize the masses of beams for drawing of initial structure
mb_init = ones(nB)*100;

# Draw initial truss structure
drawTruss(J,B,Jc,Jl,mb_init/10)

## The Optimization Problem

In lecture 5, slide 15, we learned that the weight of the truss structure defined above can be minimized by solving the problem under the constraint of a given load of 2000 kg distributed evenly on the joints on the street level: 

  \begin{equation*}
    \begin{aligned}
    \min_{\substack{f_1, \dots, f_n\\ \tilde f_1,\dots, \tilde f_n}} \quad & \sum_{b=1}^n \Big[ \delta \cdot  \|\vec d_b\| \cdot \tilde f_b  \Big]\\
    \text{subject to} \quad & 
    \vec l_j + \sum_{b \in B_j} \Big[ \frac{\vec d_b}{\|\vec d_b\|} f_b \Big] = \vec 0, & j = 1,\dots,m\\
    & \tilde f_b \geq f_b,\quad \tilde f_b \geq -f_b, & b = 1,\dots,n
    \end{aligned}
  \end{equation*}
  
This problem neglects the weight of the beams. Modify the problem to include the weight of the beams into the equality constraints. Use $\delta = 5\cdot 10^{-4}\,$kg/(Nm).
**Hint:** The gravitational force $f$ in the unit Newton depends on the mass $m$ by $f = 9.81 m\,$ and acts only into the negative y-direction.
Implement the problem as a linear program of the form:
\begin{align}
 \min_{\vec{x}} \quad & \vec{c}^\top \vec{x}\\
 \text{subject to} \quad & A\vec{x} = \vec{b}\\
 \quad & C\vec{x} \leq \vec{d},
\end{align}

with the target variable $x = [f_1, f_2, \dots, \tilde f_1, \tilde f_2, \dots]$. 

There are 2 * nB inequality constraints and  2 * (nJ - nC) equality constraints.

In [17]:
delta = 5e-4

# ================================================================================================
# Construct the vector c.
c = vcat(zeros(nB), delta*[LinearAlgebra.norm(J[b[1]]-J[b[2]]) for b in B]);
# c = vcat(zeros(nB), [LinearAlgebra.norm(J[b[1]]-J[b[2]]) for b in B]);
# ================================================================================================

In [18]:
println("Dimensions of the problem: ", length(c))
println("Inequalities: ", 2*nB)
println("Equality constraints: ", 2*(nJ-nC))

Dimensions of the problem: 862
Inequalities: 862
Equality constraints: 240


In [19]:
# ================================================================================================
# Construct matrix A and vector b for the equality constraints.
A = zeros(2 * (nJ-nC), length(c));
b = zeros(2 * (nJ-nC))

idx = 0
# iterate over all joints
for j=1:nJ
    
    # ignore cantilever joints
    if j in Jc
        continue
    end
    
    # Construct the vector b.
    # External load applied at the joint
    b[2*idx+1] = -Jl[j][1] # joint load in x-direction
    b[2*idx+2] = -Jl[j][2] # joint load in y-direction
    
    # Iterate over all beams connected to joint j
    for b in Bj[j]
        db = J[B[b][1]] - J[B[b][2]] # beam direction vector
        
        # Construct matrix A
        db_dir = db / LinearAlgebra.norm(db)
        A[2*idx+1,b] = db_dir[1] # force across the beam in x-direction --> odd rows of A
        A[2*idx+2,b] = db_dir[2] # force across the beam in y-direction --> even rows of A
        
    end
    idx = idx + 1
end
# ================================================================================================

In [20]:
# ================================================================================================
# construct matrix C and vector d for the inequality constraints Cx <= d
C = zeros(2 * nB, length(c))
d = zeros(2 * nB);

# Fill the matrix C
for b = 1:nB
    # fb - tildefb <= 0 --> odd rows of C
    C[2*b-1,b] = 1.0
    C[2*b-1,b+nB] = -1.0
    
    # -fb - tildefb <= 0 --> even rows of C
    C[2*b,b] = -1.0
    C[2*b,b+nB] = -1.0
end
# ================================================================================================

## Use the solver to find the solution

In [21]:
model = Model(SCS.Optimizer)
@variable(model, x[1:2*nB])
@constraint(model, A * x .== b)
@constraint(model, C * x .<= d)
@objective(model, Min, c'*x)
optimize!(model)

------------------------------------------------------------------
	       SCS v3.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 862, constraints m: 1102
cones: 	  z: primal zero / dual free vars: 240
	  l: linear vars: 862
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 2958, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|7.65e+004 5.41e+003 1.02e+010 5.10e+009 1.00e-001 3.13e-003 
   250|3.74e+002 7.07e-004 6.93e+002 1.65e+00

In [22]:
sol = value.(x)
masses = c.*sol
masses = masses[nB+1:2*nB]

drawTruss(J,B,Jc,Jl, masses)